In [3]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torchmetrics import Accuracy, Precision, Recall
from torchvision import datasets
import torchvision.transforms as transforms

In [8]:
train_transforms = transforms.ToTensor()
test_transforms = transforms.ToTensor()

In [9]:
# Load datasets
from torchvision import datasets
import torchvision.transforms as transforms

train_data = datasets.FashionMNIST(root='./data', train=True, download=True, transform=train_transforms)

test_data = datasets.FashionMNIST(root='./data', train=False, download=True, transform=test_transforms)

batch_size = 32

train_loader = DataLoader(train_data, batch_size = batch_size, shuffle = True)
test_loader = DataLoader(test_data, batch_size = batch_size, shuffle = True)


In [53]:

# Get the number of classes
classes = train_data.classes
num_classes = len(train_data.classes)

# Define some relevant variables
num_input_channels = 1
num_output_channels = 16
image_size = train_data[0][0].shape[1]

print(image_size)


28


In [61]:
print(num_classes)

10


In [34]:
train_data[0][0].shape[1]

28

In [6]:
# Define CNN
class MultiClassImageClassifier(nn.Module):
  
    # Define the init method
    def __init__(self, num_classes):
        super(MultiClassImageClassifier, self).__init__()
        self.conv1 = nn.Conv2d(num_input_channels, num_output_channels, kernel_size=3, stride=1, padding=1)
        self.relu = nn.ReLU()
        self.maxpool = nn.MaxPool2d(kernel_size=2, stride=2)
        self.flatten = nn.Flatten()
        
        # Create a fully connected layer
        self.fc = nn.Linear(num_output_channels * (image_size//2)**2, num_classes)
        
    def forward(self, x):
        # Pass inputs through each layer
        x = self.conv1(x)
        x = self.relu(x)
        x = self.maxpool(x)
        x = self.flatten(x)
        x = self.fc(x)
        return x

In [11]:
# Define the training set DataLoader
dataloader_train = DataLoader(
    train_data,
    batch_size=10,
    shuffle=True,
)

# Define training function
def train_model(optimizer, net, num_epochs):
    num_processed = 0
    criterion = nn.CrossEntropyLoss()
    for epoch in range(num_epochs):
        running_loss = 0
        num_processed = 0
        for features, labels in dataloader_train:
            optimizer.zero_grad()
            outputs = net(features)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()
            num_processed += len(labels)
        print(f'epoch {epoch}, loss: {running_loss / num_processed}')
        
    train_loss = running_loss / len(dataloader_train)


In [13]:
net = MultiClassImageClassifier(num_classes)
optimizer = optim.Adam(net.parameters(), lr=0.001)


# Test the model on the test set
# Define the test set DataLoader
dataloader_test = DataLoader(
    test_data,
    batch_size=10,
    shuffle=False,
)


In [12]:
 #Train for 1 epoch


train_model(
    optimizer=optimizer,
    net=net,
    num_epochs=1,
)



epoch 0, loss: 0.04021633486400824


In [44]:
print(test_data.data.shape)

print(test_data.classes)


print(test_data.training_file)

torch.Size([10000, 28, 28])
['T-shirt/top', 'Trouser', 'Pullover', 'Dress', 'Coat', 'Sandal', 'Shirt', 'Sneaker', 'Bag', 'Ankle boot']
training.pt


In [51]:
dataiter = iter(dataloader_test)
features, labels = next(dataiter)

print(f"Shape of features: {features.shape}")
print(f"Shape of labels: {labels.shape}")

Shape of features: torch.Size([10, 1, 28, 28])
Shape of labels: torch.Size([10])


In [16]:
# Define the metrics
accuracy_metric = Accuracy(task='multiclass', num_classes=num_classes)
precision_metric = Precision(task='multiclass', num_classes=num_classes, average=None)
recall_metric = Recall(task='multiclass', num_classes=num_classes, average=None)

# Run model on test set
net.eval()
predicted = []
for i, (features, labels) in enumerate(dataloader_test):
    output = net.forward(features.reshape(-1, 1, image_size, image_size))
    predicted.extend(cat.tolist())
    cat = torch.argmax(output, dim=-1)
    accuracy_metric(cat, labels)
    precision_metric(cat, labels)
    recall_metric(cat, labels)  

# Compute the metrics
accuracy = accuracy_metric.compute().item()
precision = precision_metric.compute().tolist()
recall = recall_metric.compute().tolist()
print('Accuracy:', accuracy)
print('Precision (per class):', precision)
print('Recall (per class):', recall)


Accuracy: 0.18400000035762787
Precision (per class): [0.2800000011920929, 0.5498981475830078, 0.19665709137916565, 0.0015748031437397003, 0.0, 0.026229508221149445, 0.0, 0.012738853693008423, 0.024390242993831635, 0.17574691772460938]
Recall (per class): [0.007000000216066837, 0.5400000214576721, 0.753000020980835, 0.0010000000474974513, 0.0, 0.024000000208616257, 0.0, 0.004000000189989805, 0.010999999940395355, 0.5]


In [60]:
print(cat)

tensor([9, 2, 2, 2, 1, 9, 1, 9, 7, 2])


In [59]:
output[0]

tensor([ 0.0288,  0.0481,  0.0486, -0.2247, -0.3856,  0.0668, -0.1302, -0.2045,
        -0.1212,  0.2028], grad_fn=<SelectBackward0>)